In [1]:
%pip install pymilvus

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from pymilvus import connections

# If using Docker standalone Milvus
connections.connect("default", host="127.0.0.1", port="19530")

In [4]:
from pymilvus import db
from pymilvus import Collection, FieldSchema, CollectionSchema, DataType

# 1. Create a new database
db.create_database("rag_db")

# 2. Switch to that database
db.using_database("rag_db")

# ----- Create schema -----
fields = [
    FieldSchema("doc_id", DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema("title", DataType.VARCHAR, max_length=200),
    FieldSchema("domain", DataType.VARCHAR, max_length=100),
    FieldSchema("content", DataType.VARCHAR, max_length=2000),
    FieldSchema("embedding", DataType.FLOAT_VECTOR, dim=384) 
]

schema = CollectionSchema(fields, description="Policy documents with embeddings")
collection = Collection("policy_docs_collection", schema)

# ----- Create index -----
index_params = {
    "index_type": "IVF_FLAT",
    "metric_type": "COSINE",
    "params": {"nlist": 128},
}
collection.create_index(field_name="embedding", index_params=index_params)

Status(code=0, message=)

In [5]:
# ----- Example data -----
content_chunks = [
    {
        "doc_id": 1,
        "section": "Pay Policies",
        "title": "Employee Pay Policy",
        "domain": "Human Resources",
        "content": "Employees are paid bi-weekly via direct deposit."
    },
    {
        "doc_id": 1,
        "section": "Leave of Absence",
        "title": "Leave Request and Approval Process",
        "domain": "Human Resources",
        "content": "Employees must submit a leave request for approval."
    },
    {
        "doc_id": 1,
        "section": "Internet Use",
        "title": "Acceptable Use of Company Internet",
        "domain": "IT & Security",
        "content": "Company internet must be used for work-related tasks only."
    },
    {
        "doc_id": 2,
        "section": "Break at Work",
        "title": "Employee Break Policy",
        "domain": "Workplace Operations",
        "content": "Employees can take an hour break."
    },
    {
        "doc_id": 2,
        "section": "Harassment",
        "title": "Workplace Harassment Policy",
        "domain": "Compliance",
        "content": "Interact with each employee with respect."
    }
]


# content_chunks_list = []
# for chunk in content_chunks:
#     content_chunks_list.append(chunk["content"])
content_chunks_list = [chunk["content"] for chunk in content_chunks]
print(content_chunks_list)
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

doc_vectors = model.encode(content_chunks_list)
doc_vectors.shape

['Employees are paid bi-weekly via direct deposit.', 'Employees must submit a leave request for approval.', 'Company internet must be used for work-related tasks only.', 'Employees can take an hour break.', 'Interact with each employee with respect.']


(5, 384)

In [6]:
# ---- Build columnar data ----
doc_ids = [int(i + 1) for i in range(len(content_chunks))]             # INT64
titles = [str(doc["title"]) for doc in content_chunks]                 # VARCHAR
domains = [str(doc["domain"]) for doc in content_chunks]               # VARCHAR
content = [str(doc["content"]) for doc in content_chunks]               # VARCHAR
embeddings = [list(map(float, vec)) for vec in doc_vectors]       # FLOAT_VECTOR(768)


# ---- Insert column-wise ----
collection.insert([doc_ids, titles, domains, content, embeddings])
collection.flush()

print(f"Successfully inserted {len(doc_ids)} documents into Milvus.")

Successfully inserted 5 documents into Milvus.


In [7]:
#Load the collection before searching or querying
collection.load()
res = collection.query(expr="doc_id > 0", output_fields=["doc_id", "title", "domain", "content", "embedding"], limit=5)
print(res)

data: ["{'doc_id': 1, 'title': 'Employee Pay Policy', 'domain': 'Human Resources', 'content': 'Employees are paid bi-weekly via direct deposit.', 'embedding': [0.024725129827857018, -0.009081486612558365, 0.038871243596076965, 0.03230155259370804, -0.05890551581978798, 0.03912464156746864, 0.02601800113916397, -0.04369090497493744, -0.019160067662596703, -1.1149180636493838e-06, 0.013653440400958061, 0.004326797090470791, -0.05378558859229088, 0.006868943106383085, 0.007638184353709221, -0.060696396976709366, -0.03760217875242233, 0.007140937261283398, 0.13387173414230347, 0.00173534348141402, -0.0030755633488297462, -0.07781299203634262, -0.08451439440250397, -0.0032163579016923904, 0.13382798433303833, -0.0364358015358448, 0.016922777518630028, 0.04861479625105858, -0.04028797522187233, 0.01967672072350979, -0.04371289908885956, 0.05151139944791794, -0.01629958488047123, -0.039059173315763474, -0.004453874193131924, 0.00047446342068724334, -0.03120514750480652, 0.048956215381622314, 

In [8]:
# print(utility.has_collection("demo_collection"))

# # Get details about a specific collection
# # Get collection details
# collection = Collection("demo_collection")  # instantiate the collection object
print(collection.schema)                    # show the schema
print(collection.num_entities)              # number of entities
print(collection.description)               # optional

{'auto_id': False, 'description': 'Policy documents with embeddings', 'fields': [{'name': 'doc_id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'title', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 200}}, {'name': 'domain', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 100}}, {'name': 'content', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 2000}}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 384}}], 'enable_dynamic_field': False}
5
Policy documents with embeddings


In [9]:
# Display results
for record in res:
    print(f"Doc ID: {record['doc_id']}")
    print(f"Title: {record['title']}")
    print(f"Domain: {record['domain']}")
    print(f"Content: {record['content']}")
    # Show only first 5 embedding values for readability
    print(f"Embedding (first 5): {record['embedding'][:5]}")
    print("-" * 80)

Doc ID: 1
Title: Employee Pay Policy
Domain: Human Resources
Content: Employees are paid bi-weekly via direct deposit.
Embedding (first 5): [0.024725129827857018, -0.009081486612558365, 0.038871243596076965, 0.03230155259370804, -0.05890551581978798]
--------------------------------------------------------------------------------
Doc ID: 2
Title: Leave Request and Approval Process
Domain: Human Resources
Content: Employees must submit a leave request for approval.
Embedding (first 5): [0.03315500542521477, 0.048533860594034195, 0.047362715005874634, 0.025270435959100723, 0.059312064200639725]
--------------------------------------------------------------------------------
Doc ID: 3
Title: Acceptable Use of Company Internet
Domain: IT & Security
Content: Company internet must be used for work-related tasks only.
Embedding (first 5): [-0.07135912775993347, -0.030664708465337753, 0.031837716698646545, -0.07750100642442703, -0.005032460205256939]
-------------------------------------------

In [10]:
query = "What’s the leave policy?"
query_vector = model.encode([query])[0]
query_vector[:5]  # Show only first 5 values

array([0.06045818, 0.03882852, 0.01172258, 0.03124589, 0.12124217],
      dtype=float32)

In [15]:
# Search for closest match only in the 'Human Resources' domain
results = collection.search(
    data=[query_vector],
    anns_field="embedding",
    param={"metric_type": "COSINE", "params": {"nprobe": 10}},
    limit=3,
    expr='domain == "Human Resources"',
    output_fields=["doc_id", "title", "domain", "content"]
)

context_sring = ""
for res in results[0]:
    print(f"doc_id={res.entity.get('doc_id')}, "
          f"title={res.entity.get('title')}, "
          f"domain={res.entity.get('domain')}, "
          f"content={res.entity.get('content')}, "
          f"score={res.distance}")
    context_sring += f"\n -- \n {res.entity.get('content')} " # Append content to context string

print("\nContext String for RAG:\n", context_sring)   

doc_id=2, title=Leave Request and Approval Process, domain=Human Resources, content=Employees must submit a leave request for approval., score=0.61988365650177
doc_id=1, title=Employee Pay Policy, domain=Human Resources, content=Employees are paid bi-weekly via direct deposit., score=0.2320510745048523

Context String for RAG:
 
 -- 
 Employees must submit a leave request for approval. 
 -- 
 Employees are paid bi-weekly via direct deposit. 


In [19]:
%reload_ext autoreload
%autoreload 2
import importlib
import llm_utility
importlib.reload(llm_utility)

from llm_utility import ask_question_open_ai 

query = "What’s the leave policy?"
response = ask_question_open_ai(query, context_sring)
response

'Based on the provided context, the leave policy is: Employees must submit a leave request for approval.'

In [20]:
print(f"User query: {query}")
print(f"Context: {context_sring}")

print(f"\n\nOpen AI Response: {response}")

User query: What’s the leave policy?
Context: 
 -- 
 Employees must submit a leave request for approval. 
 -- 
 Employees are paid bi-weekly via direct deposit. 


Open AI Response: Based on the provided context, the leave policy is: Employees must submit a leave request for approval.
